In [2]:
from safetensors.torch import load_file
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
#from peft import LoraConfig, get_peft_model
from sagemaker.pytorch import PyTorch
import numpy as np
import torch.nn as nn
import shutil

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from safetensors.torch import load_file
import torch
from sagemaker.serve.mode.function_pointers import Mode
from transformers import AutoTokenizer, AutoModelForCausalLM
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.serve import InferenceSpec
import boto3
from sagemaker.serve import CustomPayloadTranslator
import torch
from torchvision.transforms import transforms
from sagemaker.serve.spec.inference_spec import InferenceSpec
from transformers import pipeline
from sagemaker.serve import ModelServer
from sagemaker.pytorch import PyTorchModel
import sagemaker
from pathlib import Path
import numpy as np
import io
import os
sagemaker_session = Session()
region = boto3.Session().region_name

# get execution role
# please use execution role if you are using notebook instance or update the role arn if you are using a different role
execution_role = get_execution_role() if get_execution_role() is not None else "your-role-arn"

2025-02-23 01:07:08.600190: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 01:07:08.612749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-23 01:07:08.630003: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-23 01:07:08.635347: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 01:07:08.647631: I tensorflow/core/platform/cpu_feature_guar

In [4]:
#%%capture
#!pip install -U accelerate

In [5]:
#import unsloth

In [6]:
SAVED_MODEL = "Alexis-Az/Story-Generation-LlaMA-3.1-8B-10k"
INFER_MODEL= "Alexis-Az/Story-Generation-Model"
max_seq_length = 1024

## Loading the Finetuned Model
SAVED_MODEL is the lora huggingface repo of the model that has been fine-tuned. INFER_MODEL is the huggingface repo of the model that has the lora weights merged with the base weights for inferring.

In [ ]:
#adapter_model, tokenizer = unsloth.FastLanguageModel.from_pretrained(SAVED_MODEL, load_in_4bit=True)

In [ ]:
#adapter_model.save_pretrained_merged("Story-Generation-LlaMA-3.1-8B-10k", tokenizer)

In [ ]:
#adapter_model

In [8]:
!ls

'Finetuning Story Generation Model.ipynb'   inference_container
 README.md				    load_data.ipynb
 feature_engineering.ipynb		    sample_input.txt
 format_model.ipynb			    unsloth_compiled_cache
 hf_auth.ipynb


## Saving the Merged Model 
The model will be pushed to the INFER_MODEL huggingface repo, and will also be saved as a model .pth file for use in the container for inferrence.

In [ ]:
pth_model = AutoModelForCausalLM.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k")

In [ ]:
pth_model.push_to_hub(
    INFER_MODEL,
    tokenizer=tokenizer,
    safe_serialization=True,
    create_pr=True,
    max_shard_size="3GB",
)

In [ ]:
tokenizer.push_to_hub(INFER_MODEL)

In [ ]:
from pathlib import Path
model_dir = "./serve"
!mkdir -p {model_dir}

In [ ]:
torch.save(pth_model.state_dict(), model_dir + '/model.pth')

In [ ]:
!ls

## Scripting the Model as a .pth TorchScript file

In [9]:
#pth_model, tokenizer = unsloth.FastLanguageModel.from_pretrained(INFER_MODEL, load_in_4bit=True)

In [10]:
#unsloth.FastLanguageModel.for_inference(pth_model)

In [11]:
#from transformers import TextStreamer

In [14]:
#from unsloth.chat_templates import get_chat_template
#tokenizer = get_chat_template(
#    tokenizer,
#    chat_template = "llama-3.1",
#    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
#)

In [15]:
#text_streamer = TextStreamer(tokenizer)
#messages = [
#    {"from": "human", "value": "tell me a story about a cute park in california"},
#]
#inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

In [16]:
#_ = pth_model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

In [17]:
#pth_model.eval()

In [6]:
tokenizer = AutoTokenizer.from_pretrained(INFER_MODEL)

In [7]:
DUMMY_TEXT = 'tell me a story about a cute park in california'
tok_text = tokenizer.tokenize(DUMMY_TEXT)
tokens = tokenizer.encode(DUMMY_TEXT)
tok_arr = np.asarray(tokens).reshape(1, -1).astype(np.int32)
tok_arr = np.pad(tok_arr, [(0,0), (0, 1024-tok_arr.shape[1])])
tok_arr = np.repeat(tok_arr, 2, axis=0)

In [15]:
class TritonWrapper(nn.Module):
    
    def __init__(self, model_name: str):
        super().__init__()

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torchscript=True,
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            device_map='auto'
            )
        #self.model = torch.nn.DataParallel(self.model)
        #self.model.to('cuda:0')
        self.model.output_hidden_states = False
                
    def forward(self, tokens_tensor):
        self.model.eval()
        o = self.model(tokens_tensor, output_hidden_states=False)
        return o[0]

In [16]:
model = TritonWrapper(INFER_MODEL)
model.eval()

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

TritonWrapper(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm((4096,), 

In [18]:
assert model.training == False
with torch.no_grad():
    tokens_tensor = torch.from_numpy(tok_arr).to('cuda:0')
    o = model(tokens_tensor)
    traced_model = torch.jit.trace(model, [tokens_tensor])
    print("traced_model done")
    torch.jit.save(traced_model, f"{INFER_MODEL.split('/')[-1]}.pt")

/opt/conda/lib/python3.11/site-packages/transformers/models/llama/modeling_llama.py:729: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:


traced_model done


In [9]:
model_dir = "./serve"

In [34]:
os.mkdir(model_dir)

'./serve/Story-Generation-Model.pt'

# Using Sagemaker's 'ModelBuilder' Class to Format the PyTorch Model Artifacts

## Inference Object for Handling Input/Output of Model

In [19]:
class InferenceSpec(InferenceSpec):
    def invoke(self, input_object: object, model: object):       
        with torch.no_grad():
            output = model(input_object)
        return output
        
    def load(self, model_dir: str):
        model = AutoModelForCausalLM
        model.load_state_dict(torch.load(model_dir+'/model.pth'))
        model.eval()
        return model

inf_spec = InferenceSpec()

In [20]:
torch.__version__

'2.4.1.post300'

In [22]:
instance_type = 'ml.g4dn.12xlarge'
image = image_uris.retrieve(region=region, framework='pytorch', image_scope='inference', version='2.3', base_framework_version='pytorch2.3', instance_type=instance_type)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py311


In [29]:
value: str = "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:"
schema = SchemaBuilder(value,
            {"generated_text": "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\\nDaniel: Hello, Girafatron!\\nGirafatron: Hi, Daniel. I was just thinking about how magnificent giraffes are and how they should be worshiped by all.\\nDaniel: You and I think alike, Girafatron. I think all animals should be worshipped! But I guess that could be a bit impractical...\\nGirafatron: That\'s true. But the giraffe is just such an amazing creature and should always be respected!\\nDaniel: Yes! And the way you go on about giraffes, I could tell you really love them.\\nGirafatron: I\'m obsessed with them, and I\'m glad to hear you noticed!\\nDaniel: I\'"})

In [10]:
local_model_dir = str(Path(model_dir).resolve())

In [7]:
package_dir = 'model.tar.gz'

In [36]:
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    inference_spec=inf_spec,
    model_path=model_dir,
    role_arn=execution_role,
    image_uri=image,
    schema_builder=schema,
    env_vars={"HF_TASK":"text-generation"},
    model_server=ModelServer.TORCHSERVE
)

model_builder.build()

In [24]:
model_dir

'./serve'

In [18]:
!sudo apt install pigz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pigz is already the newest version (2.6-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [33]:
model_dir

'./serve'

In [37]:
os.chdir(model_dir)

In [39]:
import subprocess

def create_tar_pigz(tar_filename, source_dir):
    # = f"tar -I pigz -cf {tar_filename} {source_dir}"
    #subprocess.run(command, shell=True, check=True)
    command = f"tar -I pigz -cf {tar_filename} {source_dir}"
    subprocess.run(command, shell=True, check=True) 


# packaging the model file itself as the top level 'directory'
create_tar_pigz('../'+package_dir, '.')

In [41]:
os.chdir('..')

In [42]:
import boto3

# Create an S3 client
s3 = boto3.client('s3', region_name=region)  # Replace 'YOUR_REGION' with your bucket's region

bucket_name = 'unsloth-finetuned'      # Name of your S3 bucket

try:
    # Upload the file
    s3.upload_file(package_dir, bucket_name, package_dir)
except Exception as e:
    print(f"Error uploading file: {e}")
